### **Build Audio model**

In [ ]:
# install datasets packages
!pip install datasets==2.12.0
#evaluate
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00


In [ ]:
# import libaries
import io
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from xml.etree.ElementTree import parse
from xml.etree.ElementTree import fromstring
import jieba
import matplotlib.pyplot as plt
import random
import os
import torch
#import pacakges
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_io as tfio

In [ ]:
!pip install pydub

In [ ]:
# Read text and timestamps in xml files
def XMLreader_timestamps(filename=''):
  tree = parse(filename)
  root = tree.getroot()
  inv_file=[]
  par_file=[]
  inv_dict={}
  par_dict={}
  j=0
  key=str()
  for elem in root:
    for child in elem.findall('{http://www.talkbank.org/ns/talkbank}wor'):
      for gchild in child:
        if elem.attrib.get('who')=='INV':
          if gchild.text!=None:
            key=gchild.text
          if gchild.attrib!={}:
            start=gchild.attrib.get('start')
            end=gchild.attrib.get('end')
            if start or end!=None:
              if key in inv_dict:
                  key=key+str(j)
                  j=j+1
              time=(float(start), float(end))
              inv_dict[key]=time
        else:
          if gchild.text!=None:
            key=gchild.text
          if gchild.attrib!={}:
            start=gchild.attrib.get('start')
            end=gchild.attrib.get('end')
            if start or end!=None:
              time=(float(start), float(end))
              if key in par_dict:
                  key=key+str(j)
                  j=j+1
              par_dict[key]=time
    # Only included len(inv_dict)>2 in shorts_removed versions
    if inv_dict !={}: #and len(inv_dict)>2:
      inv_file.append(inv_dict)
      inv_dict={}
    elif par_dict !={}: #and len(par_dict)>2:
      par_file.append(par_dict)
      par_dict={}
  return inv_file, par_file
# Read the text +timestamps
def read_text_file_timestamps(file_path):
 os.chdir(file_path)
 inv_final_file=[]
 par_final_file=[]
 # Iterate through all files
 for file in os.listdir(file_path):
    if os.path.isdir(file)==True:
      file_path_new=file_path+"/"+file
      for file in os.listdir(file_path_new):
        if file.startswith("."):
            continue
        if file.endswith(".xml"):
            document = f"{file_path_new}"+"/"+file
            name=Path(document).stem
        # Call a read text-file function
            with open(document, 'r', encoding="utf8", errors="ignore") as f:
               inv, par=XMLreader_timestamps(f)
               inv_final_file.append(inv)
               par_final_file.append(par)
    # Check whether file is in text format or not
    else:
        if file.startswith("."):
            continue
        if file.endswith(".xml"):
            document = f"{file_path}"+"/"+file
            # call a read text-file function
            with open(document, 'r', encoding="utf8", errors="ignore") as f:
               inv, par=XMLreader_timestamps(f)
               inv_final_file.append(inv)
               par_final_file.append(par)
 return inv_final_file, par_final_file

In [ ]:
from pathlib import Path
from pydub import AudioSegment
# A function that split a audio file into lengths of audio sentences (in the control group)
def split_audio_files_control (audio_file, inv_test, par_test):
 audio = AudioSegment.from_mp3(audio_file)
 name=Path(audio_file).stem
 inv_filename=[]
 par_filename=[]
 for i in range(len(inv_test)):
    if inv_test==[]:
        inv_timestamps=[]
    else:
        inv_timestamps=inv_test[i].values()
        start=list(inv_timestamps)[0][0]
        end=list(inv_timestamps)[-1][1]
        start=start*1000
        end=end*1000
        audio_chunk=audio[start:end]
        filename=f"/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Control/cookie/{name}-inv-{start}.wav"
        inv_filename.append(filename)
    # Split the original audio files into audio sentences and export them as separate files   
    '''
    for idx,t in enumerate(inv_timestamps):
    #break loop if at last element of list
        time=tuple(1000*np.array(t))
        audio_chunk=audio[time[0]:time[1]]
        audio_chunk.export(f"/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Audio_cookie_wav_shorts_removed/Control/{name}-inv-{start}.wav", format="wav")
    '''
 for i in range(len(par_test)):
    par_timestamps=par_test[i].values()
    start=list(par_timestamps)[0][0]
    end=list(par_timestamps)[-1][1]
    start=start*1000
    end=end*1000
    audio_chunk=audio[start:end]
    filename=f"/content/drive/MyDrive/Colab_Notebooks/dementia/English/dementia/English/Pitt/Control/cookie/{name}-par-{start}.wav"
    par_filename.append(filename)
    # Split the original audio files into audio sentences and export them as separate files   
    '''
    for idx,t in enumerate(par_timestamps):
    #break loop if at last element of list
        time=tuple(1000*np.array(t))
        audio_chunk=audio[time[0]:time[1]]
        audio_chunk.export(f"/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Audio_cookie_wav_shorts_removed/Control/{name}-par-{start}.wav", format="wav")
    '''
 return inv_filename, par_filename


In [ ]:
from pathlib import Path
from pydub import AudioSegment

# A function that split a audio file into lengths of audio sentences (in the dementia group)
def split_audio_files_dementia (audio_file, inv_test, par_test):
 audio = AudioSegment.from_mp3(audio_file)
 name=Path(audio_file).stem
 count=0
 inv_filename=[]
 par_filename=[]
 for i in range(len(inv_test)):
    if inv_test==[]:
        inv_timestamps=[]
    else:
        inv_timestamps=inv_test[i].values()
        start=list(inv_timestamps)[0][0]
        end=list(inv_timestamps)[-1][1]
        start=start*1000
        end=end*1000
        audio_chunk=audio[start:end]
        filename=f"/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Dementia/cookie/{name}-inv-{start}.wav"
        inv_filename.append(filename)
    # Split the original audio files into audio sentences and export them as separate files       
    '''
    for idx,t in enumerate(inv_timestamps):
    #break loop if at last element of list
        time=tuple(1000*np.array(t))
        audio_chunk=audio[time[0]:time[1]]
        audio_chunk.export(f"/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Audio_cookie_wav_shorts_removed/Control/{name}-inv-{start}.wav", format="wav")
    '''

 for i in range(len(par_test)):
    par_timestamps=par_test[i].values()
    start=list(par_timestamps)[0][0]
    end=list(par_timestamps)[-1][1]
    start=start*1000
    end=end*1000
    audio_chunk=audio[start:end]
    filename=f"/content/drive/MyDrive/Colab_Notebooks/dementia/English/dementia/English/Pitt/Dementia/cookie/{name}-par-{start}.wav"
    par_filename.append(filename)
    # Split the original audio files into audio sentences and export them as separate files   
    '''
    for idx,t in enumerate(par_timestamps):
    #break loop if at last element of list
        time=tuple(1000*np.array(t))
        audio_chunk=audio[time[0]:time[1]]
        audio_chunk.export(f"/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Audio_cookie_wav_shorts_removed/Dementia/{name}-par-{start}.wav", format="wav")
    '''
 return inv_filename, par_filename


In [ ]:
# Get all the audio, text, timestamps in the control folder in the Pitt dataset
from pathlib import Path
def get_audio_words_timestamps_control (project_folder):
 inv_final_file=[]
 par_final_file=[]
 inv_final_name=[]
 par_final_name=[]
 for file in os.listdir(project_folder):
    if file.endswith(".mp3"):
      name=Path(file).stem
      # xml files are saved beforehand
      xml_path="/content/drive/MyDrive/Colab Notebooks/dementia/English/Pitt-xml/Control/cookie/"+f"{name}.xml"
      with open(xml_path, 'r', encoding="utf8", errors="ignore") as f:
        inv, par=XMLreader_timestamps(f)
      #inv, par=XMLreader_timestamps(xml_path)
      if inv !=[]:
        inv_final_file.append(inv)
      if par !=[]:
        par_final_file.append(par)
      new_path=project_folder+file
      # Get the corresponding audio sentences
      inv_name, par_name=split_audio_files_control(new_path, inv, par)
      if inv_name !=[]:
        inv_final_name.append(inv_name)
      if par_name !=[]:
        par_final_name.append(par_name)
 return inv_final_file, par_final_file, inv_final_name, par_final_name

In [ ]:
# Get all the audio, text, timestamps in the dementia folder in the Pitt dataset
from pathlib import Path
def get_audio_words_timestamps_dementia (project_folder):
 inv_final_file=[]
 par_final_file=[]
 inv_final_name=[]
 par_final_name=[]
 for file in os.listdir(project_folder):
    if file.endswith(".mp3"):
      name=Path(file).stem
      # Xml files are saved beforehand
      xml_path="/content/drive/MyDrive/Colab Notebooks/dementia/English/Pitt-xml/Dementia/cookie/"+f"{name}.xml"
      with open(xml_path, 'r', encoding="utf8", errors="ignore") as f:
        inv, par=XMLreader_timestamps(f)
      #inv, par=XMLreader_timestamps(xml_path)
      if inv !=[]:
        inv_final_file.append(inv)
      if par !=[]:
        par_final_file.append(par)
      new_path=project_folder+file
      # Get the corresponding audio sentences
      inv_name, par_name=split_audio_files_dementia(new_path, inv, par)
      if inv_name !=[]:
        inv_final_name.append(inv_name)
      if par_name !=[]:
        par_final_name.append(par_name)
 return inv_final_file, par_final_file, inv_final_name, par_final_name

In [ ]:
inv_control, par_control, inv_control_name, par_control_name=get_audio_words_timestamps_control('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Control/cookie/')

In [ ]:
inv_dementia, par_dementia, inv_dementia_name, par_dementia_name=get_audio_words_timestamps_dementia('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Dementia/cookie/')

In [ ]:
inv_control_name = [item for sublist in inv_control_name for item in sublist]
par_control_name = [item for sublist in par_control_name for item in sublist]
inv_dementia_name = [item for sublist in inv_dementia_name for item in sublist]
par_dementia_name = [item for sublist in par_dementia_name for item in sublist]

In [ ]:
all_filenames=inv_control_name+par_control_name+inv_dementia_name+par_dementia_name

In [ ]:
# Remove digits in sentences
def digit_remover(sentence):
 one_sentence=[]
 for word in sentence:
      new_word = ''.join((x for x in word if not x.isdigit()))
      one_sentence.append(new_word)
 return one_sentence

In [ ]:
timestamps=[]
words=[]
label=[]
# Remove the digits in the text data, normalize timestamps of sentences to start from 0:00, add labels to sentences
for item1 in inv_control:
  for item2 in item1:
    all_keys=list(item2.keys())
    words.append(digit_remover(all_keys))
    time=list(item2.values())
    first=time[0][0]
    final_time=np.array(time)-first
    timestamps.append(final_time)
    label.append(0)
# Participants in the control folder are also seen as non-dementia patients
for item1 in par_control:
  for item2 in item1:
    all_keys=list(item2.keys())
    words.append(digit_remover(all_keys))
    time=list(item2.values())
    first=time[0][0]
    final_time=np.array(time)-first
    timestamps.append(final_time)
    label.append(0)
for item1 in inv_dementia:
  for item2 in item1:
    all_keys=list(item2.keys())
    words.append(digit_remover(all_keys))
    time=list(item2.values())
    first=time[0][0]
    final_time=np.array(time)-first
    timestamps.append(final_time)
    label.append(0)
for item1 in par_dementia:
  for item2 in item1:
    all_keys=list(item2.keys())
    words.append(digit_remover(all_keys))
    time=list(item2.values())
    first=time[0][0]
    final_time=np.array(time)-first
    timestamps.append(final_time)
    label.append(1)

In [ ]:
# Process text data
def cleaner (words):
 clean_word=[]
 for sentence in words:
    sentence= [word.lower() for word in sentence]
    sentence=[word.replace("'", "") for word in sentence]
    sentence=[word.lower().replace("'", "") for word in sentence]
    sentence=[word.lower().replace("'", "").replace("_", "") for word in sentence]
    sentence=[word.lower().replace("'", "").replace("_", "").replace('in', "ing") if word.endswith('in') and (word!='in') else word for word in sentence ]
    sentence=["lotsof" if (word=="a_lot_of") or (word=="a_lot_o") else word for word in sentence ]
    #sentence=["overflowing" if word in "overflowning" else word for word in sentence ]
    #sentence=["overrun" if word in "overrunning" else word for word in sentence]
    sentence=["stool" if word == "stepstool" else word for word in sentence]
    sentence=["spigot" if word == "spigots" else word for word in sentence ]
    sentence=["raise" if (word == "upraise") or (word=='upraised') else word for word in sentence]
    sentence=["shoes" if word == "slipon" else word for word in sentence]
    clean_word.append(sentence)
 return clean_word

In [ ]:
clean_word=cleaner(words)

In [ ]:
# Perform data augmentation using Synonym Replacement 
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
# Find the synonyms in sentences
def get_synonyms(word):
    """
    Get synonyms of a word
    """
    synonyms = set()

    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym)

    if word in synonyms:
        synonyms.remove(word)

    return list(synonyms)

In [ ]:
import nltk
nltk.download('stopwords')
# Replace words with synonyms in sentences
from nltk.corpus import stopwords
def synonym_replacement(sentence, n):
    new_sentences=[]
    #words = words.split()
    new_sentence = sentence.copy()
    random_word_list = list(set([word for word in sentence if word not in stopwords.words('english')]))
    #random.shuffle(random_word_list)
    num_replaced = 0

    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
          for synonym in synonyms:
              synonym=synonym.split()
              if len(synonym)!=1 :
                continue
              else:
                new_sentence=[synonym[0] if word==random_word else word for word in new_sentence]
                new_sentences.append(new_sentence)
                new_sentence=sentence
                num_replaced += 1
                if num_replaced >= n: #only replace up to n words
                  num_replaced=0
                  break
                if  n >= len(synonyms):
                  break

    #sentence = ' '.join(new_sentence)

    return new_sentences

In [ ]:
# Add augmented sentences into datasets
final_sentences=[]
final_labels=[]
final_timestamps=[]
final_filenames=[]
for i, sentence in enumerate(clean_word):
    augmented_sentences=synonym_replacement(sentence, 2)
    final_sentences.append(sentence)
    final_labels.append(label[i])
    final_timestamps.append(timestamps[i])
    final_filenames.append(all_filenames[i])
    if len(augmented_sentences) !=0:
      for sentence in augmented_sentences:
        final_sentences.append(sentence)
        final_labels.append(label[i])
        final_timestamps.append(timestamps[i])
        final_filenames.append(all_filenames[i])

In [ ]:
# Add all the data into a DataFrame
df_word = pd.DataFrame({'word':final_sentences,'time_stamps':final_timestamps, 'prediction':final_labels, 'filenames': final_filenames})

In [ ]:
dict_temp={'word':word, 'time_stamps':time, 'predictions':label}

NameError: ignored

In [ ]:
# Create a tokenizer using current text data and tokenize the text data
# Add padding in text and timestamps data
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
tokenizer = Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(final_sentences)
word = df_word['word'].tolist()
#word_train=np.asarray(word_train).astype('float32')
time_stamps=df_word['time_stamps'].tolist()
word = tokenizer.texts_to_sequences(word)
word = tf.keras.preprocessing.sequence.pad_sequences(word, padding='post', maxlen=50, dtype=object).astype('float32')
time_stamps = tf.keras.preprocessing.sequence.pad_sequences(time_stamps, padding='post', maxlen=50, dtype=object).astype('float32')

In [ ]:
df_word['word']=list(word)

In [ ]:
df_word['time_stamps']=list(time_stamps)

In [ ]:
df_word['filenames']=all_filenames

In [ ]:
data_time=list(df_word['time_stamps'])
data_word=list(df_word['word'])

In [ ]:
from datasets import Dataset
ds_word = Dataset.from_dict({'filename': all_filenames, 'word': data_word, "time_stamps": data_time})

In [ ]:
y=df_word['prediction']

In [ ]:
df_all=pd.concat([df_audio, df_word], axis=1)

In [ ]:
df_all.rename(columns={"prediction": "label"}, inplace=True)

In [ ]:
from datasets import Audio, dataset_dict, concatenate_datasets, list_metrics

In [ ]:
# read audio sentences
final_data=load_dataset('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/Audio_cookie_wav_original', name="en-US")

FileNotFoundError: ignored

In [ ]:
# Load the Wa2vec featrue extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
# Cast column to feature for decoding.
final_data = final_data.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
# Extract "audio" array and resample the dataset to 16kHz for using the pre-trained Wav2vec model
# Add file paths to the dataset for concatenating audio data with text and timestamps data
def preprocess_function(examples):
     audio_arrays = [x["array"] for x in examples["audio"]]
     inputs = feature_extractor(
            audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True, padding=True
     )
     inputs['path']=[x['path'] for x in examples['audio']]
     return inputs

In [ ]:
processed_final_data = final_data.map(preprocess_function, batched=True, writer_batch_size=2, batch_size=2)

Map:   0%|          | 0/7686 [00:00<?, ? examples/s]

In [ ]:
# Save the processed data
aaaaprocessed_final_data.save_to_disk('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/processed_final_data_shorts_removed')

Saving the dataset (0/1 shards):   0%|          | 0/7686 [00:00<?, ? examples/s]

In [ ]:
# Read the processed data
from datasets import load_from_disk
processed_final_data=load_from_disk("/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/processed_final_data_original")

In [ ]:
# Sort audio data to align with the order of text and timestamps data
from pathlib import Path
# Obtain all the file names in the processed dataset
filename1=[Path(x).stem for x in processed_final_data['train']['path']]
from datasets import Dataset
order_list = filename1

# Extract the relevant columns from the dataset
example_ids = [Path(x).stem for x in df_word['filenames']]
word_list = df_word['word']
time_list = df_word['time_stamps']


# Create a list of tuples containing the example_id, other_column, and order
data_tuples = list(zip(example_ids, word_list, time_list))

# Sort the list of tuples based on the order_list
sorted_tuples = sorted(data_tuples, key=lambda x: order_list.index(x[0]))

# Separate the sorted data into individual lists
sorted_example_ids, sorted_word, sorted_time = zip(*sorted_tuples)

# Create a new sorted dataset using the sorted columns
sorted_dataset = Dataset.from_dict({
    'filename': sorted_example_ids,
    'word': sorted_word,
    'time_stamps': sorted_time
})


In [ ]:
# In the augmented dataset, extract the input_value from processed_final_data and sort input_values to align with text and timestamps data
from pathlib import Path
filename1=[Path(x).stem for x in processed_final_data['train']['path']]
input_values_lists=[x for x in processed_final_data['train']['input_values']]

from datasets import Dataset
# Assuming 'dataset' is your input Hugging Face dataset object
order_list = filename1

# Extract the relevant columns from the dataset
example_ids = [Path(x).stem for x in df_word['filenames']]
word_list = df_word['word']
time_list = df_word['time_stamps']
label_list=df_word['prediction']

# Create a list of tuples containing the example_id, other_column, and order
data_tuples = list(zip(example_ids, word_list, time_list, label_list))

# Sort the list of tuples based on the order_list
sorted_tuples = sorted(data_tuples, key=lambda x: order_list.index(x[0]))

# Separate the sorted data into individual lists
sorted_example_ids, sorted_word, sorted_time, sorted_label= zip(*sorted_tuples)


#make duplicate datapoints
input_values_augmented = []
name_to_input_value = {name: input_value for name, input_value in zip(filename1, input_values_lists)}

for name in sorted_example_ids:
    if name in name_to_input_value:
        input_values_augmented.append(name_to_input_value[name])


# Create a new sorted dataset using the sorted columns
sorted_dataset = Dataset.from_dict({
    'filename': sorted_example_ids,
    'word': sorted_word,
    'time_stamps': sorted_time,
    'input_values':input_values_augmented,
    'label':sorted_label
})

In [ ]:
sorted_dataset.save_to_disk('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_original_augmented')

Saving the dataset (0/8 shards):   0%|          | 0/53937 [00:00<?, ? examples/s]

In [ ]:
# Concatenante non-agumented data with the audio data
from datasets import Audio, dataset_dict, concatenate_datasets, list_metrics
final_processed_data = concatenate_datasets([processed_final_data['train'], sorted_dataset], axis=1)

ValueError: ignored

In [ ]:
final_combined_data=final_processed_data

In [ ]:
# Save the combined and processed final data set
final_combined_data.save_to_disk('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_shorts_removed_version')

In [ ]:
from datasets import load_from_disk
final_combined_dataset=load_from_disk('/content/drive/MyDrive/Colab_Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_shorts_removed_version')